# Module 5: MNIST handwritten digits

In this lab, we are going to learn to recognize hand-written digits from the MNIST data set.

This dataset is the "IRIS" data of neural networks.
 * http://yann.lecun.com/exdb/mnist/

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import os, sys
import itertools, functools
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import scale, LabelBinarizer
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Random seed for numpy
np.random.seed(18937)

# Introducing Keras

Keras is an abstraction of the TensorFlow API to facilitate more easily constructed models.
And actually, it is a general Python library for model construction that support TensorFlow and some other underlying lirbaries. 
  * https://keras.io/
  

In [ ]:
from keras.models import Model
from keras.layers import Input
from keras.layers.core import Dense, Dropout, Reshape
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import Adam

In the cell below, we construct a [Convolutional Neural Network](http://deeplearning.net/tutorial/lenet.html) that has the following structure:
  * Convolution with 5x5 pixel kernels, 32 of them, and using the [Rectified Linear Unit](https://en.wikipedia.org/wiki/Rectifier_(neural_networks)
  * Max Pooling with 2x2 kernel: Find the strongest response in each 2x2 neuron area of a generated feature map (from the convolution)
     * Good Pooling Page: http://ufldl.stanford.edu/tutorial/supervised/Pooling/
  * Convolve with 64 5x5 kernels, then Max Pooling again
  * Strecth all the feature maps out into a vector
  * A feed forward, fully connected layer -- think just dense vector -- of 1024 neurons
  * 10 class activation using SoftMax, a logit layer, with all neurons normalized to sum to 1.0
    * https://en.wikipedia.org/wiki/Softmax_function
  

In [ ]:
layers = [   
    Input(shape=(784,)),
    Reshape((28, 28, 1)),
    
    # Convolutional network
    Conv2D(32, (5, 5), activation='relu', padding='SAME'),
    MaxPooling2D((2,2), strides=(2,2), padding='SAME'),
    Conv2D(64, (5, 5), activation='relu', padding='SAME'),
    MaxPooling2D((2,2), strides=(2,2), padding='SAME'),
        
    # Fully connected network
    Reshape((7*7*64,)),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax'),
]

y_pred = functools.reduce(lambda f1, f2: f2(f1), layers)

model = Model(inputs = [layers[0]], outputs = [y_pred])
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()

## Load dataset

In [ ]:
from tensorflow.examples.tutorials import mnist
dataset = mnist.input_data.read_data_sets('/dsa/data/all_datasets/MNIST_data', one_hot=True)

## Train the model

In [ ]:
model.fit(x=[dataset.train.images], y=[dataset.train.labels], batch_size=50, epochs=10,
      validation_data=(dataset.validation.images, dataset.validation.labels), shuffle=True, verbose=1)

## Evaluate the model

In [ ]:
print('Evaluation')
print('loss: %.4f  accuracy: %.4f' %
      tuple(model.evaluate(x=[dataset.test.images], y=[dataset.test.labels], batch_size=50, verbose=2)))

# Save your notebook